**IMPORTS**

In [24]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
import string

**OPEN DOCUMENT**

In [25]:
def openDocument(path):
    open(path, 'r', errors = 'ignore', encoding = '#utf-8')

In [26]:
f = openDocument('PierreInfoText.txt')

**TOKENIZATION**

In [33]:
def tokenization(doc):
    raw_doc = doc.read()
    raw_doc = raw_doc.lower()
    # nltk.download('punkt')
    # nltk.download('wordnet')
    # nltk.download('omw-1.4')
    # nltk.download('stopwords')
    return nltk.sent_tokenize(raw_doc), nltk.word_tokenize(raw_doc)

In [34]:
sent_tokens, word_tokens = tokenization(f)

AttributeError: 'NoneType' object has no attribute 'read'